# 1D vectorization

In [175]:
import quantumpropagator as qp
%load_ext Cython

Vm = np.ones((7,2))
Km = np.ones((7,9,3))*10e-8
theL = 7
nstates = 2
xl = theL*nstates
dthe = 0.003

The Cython extension is already loaded. To reload it, use:
  %reload_ext Cython


In [205]:
%%cython

import numpy as np
cimport numpy as np

def wrapper_Checkaddress1D_prime(Vm,Km,theL,nstates,dthe):
    a,b,c,d,k = Checkaddress1D(Vm,Km,theL,nstates,dthe,100000)
    return (k)

def wrapper_Checkaddress1D(Vm,Km,theL,nstates,dthe, k):
    a,b,c,d,k = Checkaddress1D(Vm,Km,theL,nstates,dthe, k)
    return (np.asarray(a), np.asarray(b), np.asarray(c), np.asarray(d))

def transform(t,s):
    nstates = 2
    if t < 0 or t > 7-1:
        #print(t,s)
        res = -1
    else:
        res = t*nstates+s
 
    return res 

def quick_multiply(a,b,d,xl,x):
    '''
    a = values
    b = columns
    d = rows counting
    i = rows
    '''
    I = -1j
    mom = np.empty_like(x)
    for i in range(xl):
        for j in range(d[i],d[i+1]):
            mom[i] = mom[i] + a[j] * I * x[b[j]]
    return mom


cdef Checkaddress1D(double [:,:] Vm, double [:,:,:] Km, int theL, int nstates, double dthe, int kL):
    cdef:
        int s, t, k, add1
        double value1
        double [:] values_big
        int [:] i_big, j_big, d_big

    values_big = np.zeros(kL) # real
    i_big = np.zeros(kL, dtype=np.intc) #integers
    j_big = np.zeros(kL, dtype=np.intc) #integers
    d_big = np.zeros(theL*nstates+1, dtype=np.intc) #integers

    k = 0
    d_big[0] = k
    for t in range(theL):
        for s in range(nstates):
            i_add = transform(t,s)
            # derivatives in theta
            # dG_dt   = ((1.0/12)*GRID[t-2,s]+(-2.0/3)*GRID[t-1,s]+(2.0/3)*GRID[t+1,s]+(-1.0/12)*GRID[t+2,s]) / dthe
            # d2G_dt2 = (-GRID[t+2,s]+16*GRID[t+1,s]-30*GRID[t,s]+16*GRID[t-1,s]-GRID[t-2,s]) / (12 * dthe**2)

            value1 = ((1.0/12) * Km[t,8,1] / dthe) + (-(1.0/12) * Km[t,8,2] / (dthe**2))
            add1   = transform(t-2,s)
            if add1 >= 0:
                values_big[k] = value1
                i_big[k] = i_add
                j_big[k] = add1
                k = k + 1 

            value1 = (-2.0/3) * Km[t,8,1] / dthe + (+(16.0/12) * Km[t,8,2] / (dthe**2))
            add1   = transform(t-1,s)
            if add1 >= 0:
                values_big[k] = value1
                i_big[k] = i_add
                j_big[k] = add1
                k = k + 1 

            value1 = (-(30.0/12) * Km[t,8,2] / (dthe**2)) + Vm[t,s]
            add1   = transform(t,s)
            if add1 >= 0:
                values_big[k] = value1
                i_big[k] = i_add
                j_big[k] = add1
                k = k + 1 

            value1 = (2.0/3) * Km[t,8,1] / dthe + (+(16.0/12) * Km[t,8,2] / (dthe**2))
            add1   = transform(t+1,s)
            if add1 >= 0:
                values_big[k] = value1
                i_big[k] = i_add
                j_big[k] = add1
                k = k + 1 

            value1 = (-(1.0/12) * Km[t,8,1] / dthe) + (-(1.0/12) * Km[t,8,2] / (dthe**2))
            add1   = transform(t+2,s)
            if add1 >= 0:
                values_big[k] = value1
                i_big[k] = i_add
                j_big[k] = add1
                k = k + 1
            d_big[i_add+1] = k
    return(values_big, i_big, j_big, d_big, k)


In [206]:
k = wrapper_Checkaddress1D_prime(Vm,Km,theL,nstates,dthe)
print(k)

58


In [207]:
a,b,c,d = wrapper_Checkaddress1D(Vm,Km,theL,nstates,dthe,k)

In [208]:
a.shape, b.shape, c.shape, d.shape

((58,), (58,), (58,), (15,))

In [209]:
a,b,c

(array([  9.72222222e-01,   1.48370370e-02,  -9.28703704e-04,
          9.72222222e-01,   1.48370370e-02,  -9.28703704e-04,
          1.47925926e-02,   9.72222222e-01,   1.48370370e-02,
         -9.28703704e-04,   1.47925926e-02,   9.72222222e-01,
          1.48370370e-02,  -9.28703704e-04,  -9.23148148e-04,
          1.47925926e-02,   9.72222222e-01,   1.48370370e-02,
         -9.28703704e-04,  -9.23148148e-04,   1.47925926e-02,
          9.72222222e-01,   1.48370370e-02,  -9.28703704e-04,
         -9.23148148e-04,   1.47925926e-02,   9.72222222e-01,
          1.48370370e-02,  -9.28703704e-04,  -9.23148148e-04,
          1.47925926e-02,   9.72222222e-01,   1.48370370e-02,
         -9.28703704e-04,  -9.23148148e-04,   1.47925926e-02,
          9.72222222e-01,   1.48370370e-02,  -9.28703704e-04,
         -9.23148148e-04,   1.47925926e-02,   9.72222222e-01,
          1.48370370e-02,  -9.28703704e-04,  -9.23148148e-04,
          1.47925926e-02,   9.72222222e-01,   1.48370370e-02,
        

In [210]:
lol = np.zeros((14,14))

In [211]:
lol[b,c] = a

In [212]:
qp.printMatrix2D(lol)

          1         2         3         4         5         6         7   \
1   0.972222  0.000000  0.014837  0.000000 -0.000929  0.000000  0.000000   
2   0.000000  0.972222  0.000000  0.014837  0.000000 -0.000929  0.000000   
3   0.014793  0.000000  0.972222  0.000000  0.014837  0.000000 -0.000929   
4   0.000000  0.014793  0.000000  0.972222  0.000000  0.014837  0.000000   
5  -0.000923  0.000000  0.014793  0.000000  0.972222  0.000000  0.014837   
6   0.000000 -0.000923  0.000000  0.014793  0.000000  0.972222  0.000000   
7   0.000000  0.000000 -0.000923  0.000000  0.014793  0.000000  0.972222   
8   0.000000  0.000000  0.000000 -0.000923  0.000000  0.014793  0.000000   
9   0.000000  0.000000  0.000000  0.000000 -0.000923  0.000000  0.014793   
10  0.000000  0.000000  0.000000  0.000000  0.000000 -0.000923  0.000000   
11  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000 -0.000923   
12  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
13  0.000000

In [213]:
d

array([ 0,  3,  6, 10, 14, 19, 24, 29, 34, 39, 44, 48, 52, 55, 58], dtype=int32)

In [216]:
x = np.ones(14, dtype=complex)
xl = 14

In [219]:
%%time
quick_multiply(a,b,d,xl,x)

CPU times: user 389 µs, sys: 27 µs, total: 416 µs
Wall time: 425 µs


array([  4.64834067e-310 -9.86130556e-001j,
         4.64833678e-310 -9.86130556e-001j,
         4.64834081e-310 -1.00092315e+000j,
         6.91231097e-310 +7.12911385e+024j,
         0.00000000e+000 -1.00000000e+000j,
        -2.33162847e+135 -1.00000000e+000j,
         0.00000000e+000 -1.00000000e+000j,
         0.00000000e+000 -1.00000000e+000j,
         2.07347219e+248 -1.00000000e+000j,
         0.00000000e+000 -1.00000000e+000j,
         0.00000000e+000 -1.00092870e+000j,
        -2.26715792e-017 -1.00092870e+000j,
         0.00000000e+000 +1.69710714e+215j,
         0.00000000e+000 -9.86091667e-001j])